In [1]:
from apex import *
from proofs import *

ZigZag nodes: 33554432


In [2]:
z = ZigZag(security=proofs.filecoin_security_requirements, partitions=8, size=64*GiB, merkle_hash=pedersen)

In [3]:
(l1, savings, savings_, best_) = optimize(z); (l, savings, savings_, best_)

(17, 4231.21035876649, '01:10:31.2', '124:03:59.1')

In [4]:
a = apex(z, l); a

ZigZag(instance=None, circuit_proof_size=192, hash_size=32, size=68719476736, partitions=8, merkle_hash=HashFunction(hash_time=1.7993e-05, constraints=1152), kdf_hash=HashFunction(hash_time=1.0428e-07, constraints=10324), security=Security(base_degree=5, expansion_degree=8, layers=10, total_challenges=8848, sloth_iter=0), processor_power=165, relax_time=1, constraint_proving_time=5.013000000000001e-05, apex_height=17)

In [5]:
zz = ZigZag(security=proofs.filecoin_security_requirements, partitions=8, size=64*GiB, merkle_hash=blake2s)

In [6]:
(ll, savings, savings_, best_) = optimize(zz); (ll, savings, savings_, best_)

(17, 37803.11624896135, '10:30:3.1', '17:13:43.2')

In [7]:
aa = apex(zz, ll); aa

ZigZag(instance=None, circuit_proof_size=192, hash_size=32, size=68719476736, partitions=8, merkle_hash=HashFunction(hash_time=1.0428e-07, constraints=10324), kdf_hash=HashFunction(hash_time=1.0428e-07, constraints=10324), security=Security(base_degree=5, expansion_degree=8, layers=10, total_challenges=8848, sloth_iter=0), processor_power=165, relax_time=1, constraint_proving_time=5.013000000000001e-05, apex_height=17)

In [8]:
humanize_seconds(z.groth_proving_time())

'02:21:13.7'

In [9]:
humanize_seconds(a.groth_proving_time())

'01:10:55.4'

In [10]:
humanize_seconds(z.groth_proving_time()-a.groth_proving_time())

'01:10:18.2'

In [11]:
a.groth_proving_time()/z.groth_proving_time()

0.5021941289784836

In [12]:
humanize_seconds(z.total_seal_time())

'125:14:30.3'

In [13]:
humanize_seconds(a.total_seal_time())

'124:03:59.1'

In [14]:
1 - (a.total_seal_time() / z.total_seal_time())

0.009384539060641295

In [15]:
humanize_seconds(a.replication_time())

'01:54:33.2'

In [16]:
aa.merkle_hash

HashFunction(hash_time=1.0428e-07, constraints=10324)